In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

# Carrega o arquivo CSV contendo dados de repositórios
df = pd.read_csv('../Database/Resultados-Busca/repositories.csv')

# Função para contar palavras-chave em todas as descrições dos repositórios


def total_keyword_count(df, keywords):
    # Inicializa um dicionário para armazenar contagens de palavras-chave
    total_counts = {keyword: 0 for keyword in keywords}
    # Itera sobre cada descrição no DataFrame
    for description in df['description']:
        # Para cada palavra-chave, conta quantas vezes aparece na descrição
        for keyword in keywords:
            total_counts[keyword] += description.lower().count(keyword.lower())
    return total_counts

# Função para criar diretórios, caso não existam


def create_dirs(output_dir, image_dir):
    os.makedirs(output_dir, exist_ok=True)  # Cria diretório de saída
    os.makedirs(image_dir, exist_ok=True)    # Cria diretório para imagens

# Função para salvar gráficos


def save_plot(fig, filename):
    fig.tight_layout()  # Ajusta o layout do gráfico
    plt.savefig(filename, dpi=300)  # Salva o gráfico com alta resolução
    plt.close(fig)  # Fecha a figura para liberar memória

# Função para criar histogramas


def plot_histogram(data, title, xlabel, ylabel, filename):
    fig, ax = plt.subplots(figsize=(10, 6))  # Cria uma nova figura
    ax.hist(data, bins=30, alpha=0.7, color='skyblue',
            edgecolor='black')  # Plota o histograma
    # Define o título do gráfico
    ax.set_title(title, fontsize=16, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=14)  # Define o rótulo do eixo x
    ax.set_ylabel(ylabel, fontsize=14)  # Define o rótulo do eixo y
    ax.grid(axis='y', linestyle='--', alpha=0.7)  # Adiciona linhas de grade
    save_plot(fig, filename)  # Salva o gráfico

# Função para plotar gráficos de barras


def plot_bar_chart(x, y, hue, title, xlabel, ylabel, filename):
    fig, ax = plt.subplots(figsize=(12, 7))  # Cria uma nova figura
    # Usa o Seaborn para criar um gráfico de barras com uma paleta de cores
    sns.barplot(x=x, y=y, hue=hue, palette='viridis', ax=ax, legend=False)
    # Define o título do gráfico
    ax.set_title(title, fontsize=16, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=14)  # Define o rótulo do eixo x
    ax.set_ylabel(ylabel, fontsize=14)  # Define o rótulo do eixo y
    ax.tick_params(axis='x', rotation=45)  # Rotaciona os rótulos do eixo x
    ax.grid(axis='y', linestyle='--', alpha=0.7)  # Adiciona linhas de grade
    save_plot(fig, filename)  # Salva o gráfico

# Função para exportar DataFrame para um arquivo CSV


def export_to_csv(df, filename):
    # Salva o DataFrame como CSV
    df.to_csv(filename, index=False, sep=',', decimal=',')


# Palavras-chave de UX
ux_labels = {
    "Customer Experience", "CX", "User Experience",
    "Usability", "Ease of Use", "Usefulness",
    "User Interaction", "Human-Computer Interaction", "HCI", "Interaction Design",
    "User-Centered Design", "UCD", "Human-Centered Design", "HCD",
    "Interactive Experience", "Interactive Design", "User Engagement",
    "Digital Experience", "Online Experience", "Virtual Experience",
    "User Interface", "UI", "Interface Design", "UI Design",
    "Navigability", "Navigation", "User Navigation",
    "UX", "User Experience", "UX Design"
}

# Conta o total de palavras-chave nas descrições
total_counts = total_keyword_count(df, ux_labels)

# Cria as pastas 'analises' e 'Image/analise' se não existirem
output_dir = '../Database/Resultados-Busca/Primeiras-Analises'
image_dir = '../Image/Resultados-Busca/Primeiras-Analises'
create_dirs(output_dir, image_dir)

# 1. Distribuição de Stars e Forks
plot_histogram(df['stars'], 'Distribuição de Stars', 'Stars',
               'Frequência', os.path.join(image_dir, 'stars_distribution.png'))
plot_histogram(df['forks'], 'Distribuição de Forks', 'Forks',
               'Frequência', os.path.join(image_dir, 'forks_distribution.png'))

# Análise de correlação entre estrelas e forks
corr_coef = df['stars'].corr(df['forks'])

# 2. Análise por Linguagem
# Conta quantas vezes cada linguagem aparece
lang_counts = df['language'].value_counts()

# Mantém as 50 linguagens mais populares e soma as restantes como "Outros"
if len(lang_counts) > 50:
    other_count = lang_counts[50:].sum()
    lang_counts = lang_counts[:50]  # Mantém as 50 mais populares
    lang_counts['Outros'] = other_count  # Adiciona "Outros"

# Gráficos por Linguagem
plot_bar_chart(lang_counts.index, lang_counts.values, hue=None, title='Contagem de Repositórios por Linguagem',
               xlabel='Linguagem', ylabel='Contagem', filename=os.path.join(image_dir, 'repositories_by_language.png'))

# Estrelas e Forks Médios por Linguagem
# Calcula média de estrelas por linguagem
avg_stars_by_lang = df.groupby('language')['stars'].mean()
# Calcula média de forks por linguagem
avg_forks_by_lang = df.groupby('language')['forks'].mean()

# Filtra valores com menos de 100 ocorrências (descomentar se necessário)
# avg_stars_by_lang = avg_stars_by_lang[avg_stars_by_lang.index.map(
#     df['language'].value_counts()) >= 30].sort_values(ascending=False)
# avg_forks_by_lang = avg_forks_by_lang[avg_forks_by_lang.index.map(
#     df['language'].value_counts()) >= 30].sort_values(ascending=False)

# Plota os gráficos de estrelas
plot_bar_chart(avg_stars_by_lang.index, avg_stars_by_lang.values, hue=None, title='Estrelas Médias por Linguagem',
               xlabel='Linguagem', ylabel='Estrelas Médias', filename=os.path.join(image_dir, 'average_stars_by_language.png'))

# Plota os gráficos de forks
plot_bar_chart(avg_forks_by_lang.index, avg_forks_by_lang.values, hue=None, title='Forks Médios por Linguagem',
               xlabel='Linguagem', ylabel='Forks Médios', filename=os.path.join(image_dir, 'average_forks_by_language.png'))

# 3. Análise Temporal
# Converte a coluna de data de criação
creation_dates = pd.to_datetime(df['created_at'])
# Conta repositórios por ano de criação
creation_year_counts = creation_dates.dt.year.value_counts()

# Converte a coluna de data de atualização
recent_updates = pd.to_datetime(df['updated_at'])
# Conta repositórios por ano de atualização
recent_updates_counts = recent_updates.dt.year.value_counts()

# Plota gráficos de contagem de criação
plot_bar_chart(creation_year_counts.index, creation_year_counts.values, hue=None, title='Contagem de Anos de Criação',
               xlabel='Ano', ylabel='Contagem', filename=os.path.join(image_dir, 'creation_year_counts.png'))

plot_bar_chart(recent_updates_counts.index, recent_updates_counts.values, hue=None, title='Contagem de Atualizações Recentes',
               xlabel='Ano', ylabel='Contagem', filename=os.path.join(image_dir, 'recent_updates_counts.png'))

# 4. Categorização de Repositórios
# Conta repositórios por categoria
category_counts = df['category'].value_counts()
# Calcula média de estrelas por categoria
avg_stars_by_category = df.groupby('category')['stars'].mean()
# Calcula média de forks por categoria
avg_forks_by_category = df.groupby('category')['forks'].mean()

# Plota gráficos de contagem de repositórios por categoria
plot_bar_chart(category_counts.index, category_counts.values, hue=None, title='Contagem de Repositórios por Categoria',
               xlabel='Categoria', ylabel='Contagem', filename=os.path.join(image_dir, 'repositories_by_category.png'))

plot_bar_chart(avg_stars_by_category.index, avg_stars_by_category.values, hue=None, title='Estrelas Médias por Categoria',
               xlabel='Categoria', ylabel='Estrelas Médias', filename=os.path.join(image_dir, 'average_stars_by_category.png'))

plot_bar_chart(avg_forks_by_category.index, avg_forks_by_category.values, hue=None, title='Forks Médios por Categoria',
               xlabel='Categoria', ylabel='Forks Médios', filename=os.path.join(image_dir, 'average_forks_by_category.png'))

# Salva cada análise em um arquivo CSV separado
export_to_csv(pd.DataFrame({'Coeficiente de correlação entre estrelas e forks': [
              corr_coef]}), os.path.join(output_dir, 'correlation_stars_forks.csv'))
export_to_csv(lang_counts.reset_index(), os.path.join(
    output_dir, 'repositories_by_language.csv'))
export_to_csv(avg_stars_by_lang.reset_index(), os.path.join(
    output_dir, 'average_stars_by_language.csv'))
export_to_csv(avg_forks_by_lang.reset_index(), os.path.join(
    output_dir, 'average_forks_by_language.csv'))
export_to_csv(creation_year_counts.reset_index(),
              os.path.join(output_dir, 'creation_year_counts.csv'))
export_to_csv(recent_updates_counts.reset_index(),
              os.path.join(output_dir, 'recent_updates_counts.csv'))
export_to_csv(category_counts.reset_index(), os.path.join(
    output_dir, 'repositories_by_category.csv'))
export_to_csv(avg_stars_by_category.reset_index(), os.path.join(
    output_dir, 'average_stars_by_category.csv'))
export_to_csv(avg_forks_by_category.reset_index(), os.path.join(
    output_dir, 'average_forks_by_category.csv'))

# 5. Total de palavras-chave
# Cria um DataFrame para os totais de palavras-chave
keyword_counts_df = pd.DataFrame.from_dict(
    total_counts, orient='index', columns=['Total'])
# Ordena os totais de palavras-chave
keyword_counts_df.sort_values(by='Total', ascending=False, inplace=True)
# Salva os totais em um arquivo CSV
export_to_csv(keyword_counts_df.reset_index().rename(columns={'index': 'Palavra-chave'}),
              os.path.join(output_dir, 'keyword_totals.csv'))

print("Análise concluída e gráficos salvos.")

C:\Users\tochi\AppData\Local\Temp\ipykernel_10260\1804429277.py:57: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=x, y=y, hue=hue, palette='viridis', ax=ax, legend=False)
C:\Users\tochi\AppData\Local\Temp\ipykernel_10260\1804429277.py:57: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=x, y=y, hue=hue, palette='viridis', ax=ax, legend=False)
C:\Users\tochi\AppData\Local\Temp\ipykernel_10260\1804429277.py:57: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=x, y=y, hue=hue, palette='viridis', ax=ax, legend=False)
C:\Users\tochi\AppData\Local

Análise concluída e gráficos salvos.
